 # TWITTER SCRAPPING DATASETS LOADING

## TOKEN ACCESS

In [1]:
TWITTER_CONSUMER_KEY = 'IXBsOhx5JVqDH4WiScZnHvJC8'
TWITTER_CONSUMER_SECRET = '8rSBkRBy5CxE1As9S64hnHjctSPkUmZ2GnTvL3MNjoY2DTPVfi'
TWITTER_ACCESS_TOKEN = '1536977498650685440-3pCXHUtEx3m0A9MZji6yWUDQBOceUW'
TWITTER_ACCESS_TOKEN_SECRET = 'JHInPSzHxfd4tdtAp9TyfxDcjKre8H8Y0aGJczVknBoL6'
BEARER_TOKEN = 'AAAAAAAAAAAAAAAAAAAAAP0kdwEAAAAA5caCys8AnDktaOytX5szfYD%2FWms%3DsPZBH2OCUXtPR4feADG7sos4frqL9BBj3cGjivJZ0TnpYCSo4A'

## Importation of Libraries

In [5]:
!pip install tweepy

import tweepy
import time
import json
import pandas as pd
import csv

import nltk
from nltk import tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import re
import collections
from collections import Counter
import itertools 

## Authentication

In [6]:
def authenticate():
    auth = tweepy.OAuthHandler(TWITTER_CONSUMER_KEY, TWITTER_CONSUMER_SECRET)
    auth.set_access_token(TWITTER_ACCESS_TOKEN, TWITTER_ACCESS_TOKEN_SECRET)    
    api = tweepy.API(auth, wait_on_rate_limit=True)
    return api

## DICTIONNARY SET 

In [7]:
word_dictionnary = "(écologie OR énergie OR éolienne OR éléctricité OR transport OR nucléaire OR co2 OR carbonne OR petrole OR renouvelables OR énergies renouvelables OR chauffage électrique OR énergie fossiles OR gaz OR hydraulique OR climat OR solaire OR effet de serre OR énergie primaire OR eéologique OR énergitique OR ecologique OR recyclage OR biodiversite OR biomasse OR développement durable OR écocitoyen OR écosysteme) lang:fr"
word_dictionnary_wordcloud = ["écologie" , "énergie" , "éolienne" , "éléctricité" , "transport" , "nucléaire" , "co2" , "carbone" , "petrole" , "renouvelables" , "énergies renouvelables" , "chauffage" "électrique" , "énergies fossiles" , "gaz" , "hydraulique" , "climat" , "solaire" , "effet de serre" , "énergie primaire" , "écologique" , "énergitique" , "ecologique" , "recyclage" , "biodiversite" , "biomasse" , "développement durable" , "écocitoyen ","écosysteme", 'Nucléaire','CNR','Transition','Développement','Chaleur',
'énergétique','#TransitionEnergétique','IFPEN','Framatome','EDF','Energy','déchets','énergie','biomasse','#energierenouvelable','Energies','Renouvelable','l’énergie','photovoltaïque','centrales','carbone','électrique','climat' , '#COP26' ,                          'consommation','#Nucléaire','gaz']
actor_dictionnary = ["IFPENinnovation", "Ecologie_Gouv", "EU_Commission", "ARP_NUC","Framatome_", "ENGIE_EC", "EDF_Entreprises", "elonmusk", "Arkolia_EnR","EnergieCap","CNR_Officiel", "Groupe_Coriance","DalkiaWEnergy","amisdelaterre","Enercoop_SCIC", "ademe", "Novethic", "Ecologie_Gouv", "lemonde_planete", "FNEasso", "Alternatiba_", "EDFofficiel", "ecosia", "elonmusk", "GretaThunberg","Youth4Climatefr", "McKinsey_France","J_B_Levy", "PPouyanne", "sdnfr", "greenpeacefr", "WWFFrance", "JMJancovici", "nouel_juliette", "huetsylvestre", "NicolasMeilhan", "audreygarric", "valmasdel", "alaingrandjean", "theShiftPR0JECT", "youmatter_fr","IFPENinnovation", "EU_Commission", "Framatome_", "ENGIE_EC", "EDF_Entreprises", "Albioma", "Arkolia_EnR","EnergieCap","CNR_Officiel", "Groupe_Coriance","DalkiaWEnergy", 

                    "CFigueres" ,
                    "yjado" ,
                    "algore" ,
                "CCNUCC",
                   
                         "RACFrance", 
                    
                ] 

## 1ST DATASET : SEARCH TWEETS FOLLOWING THE WORD DICTIONNARY

### GETTER OF 1ST DATASET'S ATTRIBUTES

In [8]:
def getretweet(id):
    api = authenticate()
    status = api.get_status(id)
    retweet_count = status.retweet_count       
    return retweet_count

def getClient():
    client = tweepy.Client(bearer_token=BEARER_TOKEN)
    return client

def getTweet(client,id):
    tweet = client.get_tweet(id, expansions = ['author_id'], user_fields= ['username'])
    return tweet

### SEARCHTWEET OF 1ST DATABASE WITH WORD DICTIONNARY

In [9]:
results = []
client = getClient()

def searchTweetsWordDictionnary(client, query):
    
    tweets = client.search_recent_tweets(query=query, tweet_fields=['context_annotations','created_at'], max_results=100)
    tweet_data = tweets.data  
    pagination_tweets = tweepy.Paginator(client.search_recent_tweets, query=query,
                                  tweet_fields=['context_annotations', 'created_at'], max_results=100).flatten(limit=300)
    if not tweet_data is None and len(tweet_data) > 0:
        for tweet in pagination_tweets :
            twt = getTweet(client,tweet['id'])
            username= twt.includes['users'][0].username
            retweet= getretweet(tweet.id)   
            id = tweet.id
            text = tweet.text
            date = tweet.created_at        
            line = {'USER_ID' : id, 'DATE_TIME' : date, 'USERNAME' :username , 'TWEET_TEXT':text, 'NUMBER_OF_RETWEET': retweet }                     
            results.append(line)
    else:
        return ''
    return results


### PRINT IN A DATAFRAME TO EXPORT 1ST DATABASE

In [10]:
tweets = searchTweetsWordDictionnary(client, word_dictionnary)
all_tweet = pd.DataFrame(results)

#all_tweet.to_csv('First_Dataset.csv')

display(all_tweet)

,USER_ID,DATE_TIME,USERNAME,TWEET_TEXT,NUMBER_OF_RETWEET
0,1649730716140204035,2023-04-22 11:03:47+00:00,ppedrito64,@EmmanuelMacron Cessez de mentir. Sortez l’éle...,0
1,1649730714210795521,2023-04-22 11:03:47+00:00,patrick27190,"RT @ChamboncelBen: "" C'est inadmissible""\n\nLa...",25
2,1649730710758981632,2023-04-22 11:03:46+00:00,dyzatch,"RT @Europhobe: 7 milliards, c'est le prix d'un...",145
3,1649730707634237441,2023-04-22 11:03:45+00:00,Colombe_w,RT @DigitalGanon: Leurs vieilles ganache de bo...,271
4,1649730706342309888,2023-04-22 11:03:45+00:00,GalaT77,RT @Reporterre: Plus de routes ne signifie pas...,10
...,...,...,...,...,...
295,1649729672912027650,2023-04-22 10:59:39+00:00,GillardFabien,"RT @elkevdbrandt: Plus de trains 🚊 , vers et d...",17
296,1649729670252748801,2023-04-22 10:59:38+00:00,Josianels3,RT @f_philippot: Pourquoi dit-on qu’on va manq...,1679
297,1649729661755064321,2023-04-22 10:59:36+00:00,ClaireMeilleray,RT @JLMTV_INFOS: Bonjour Twitter. \nHier le mi...,33
298,1649729660383576065,2023-04-22 10:59:36+00:00,stephaneLetell7,RT @f_philippot: La seule raison pour laquelle...,343


## 2ND DATASET : SEARCH TWEETS FOLLOWING USERS ACTIVITY WITH USER DICTIONNARY

In [ ]:
results=[]
def searchTweetsUserAct(username):
    
    apiAuth = authenticate()
    tweets = apiAuth.user_timeline(screen_name = username, include_rts = False, count=10000, tweet_mode="extended" )
    
    results=[]
   
     
    if not tweets is None and len(tweets) > 0:
        for tweetInfo in tweets:
            
            tweetId = tweetInfo.id
            tweetDate = tweetInfo.created_at
            tweetCreator = tweetInfo.user.name
            tweetText = tweetInfo.full_text
            tweetFav = tweetInfo.favorite_count
            tweetRet = tweetInfo.retweet_count

            line = {'ID': tweetId, 'DATE' : tweetDate, 'CREATOR' : tweetCreator, 'TEXT' : tweetText, 'LIKE' : tweetFav, 'RETWEET' : tweetRet}
            results.append(line)
            

        return results
    else:
        return ""

    
def search_usersTweets_info():

    results_tab= []
    
    for user in actor_dictionnary:
        results_tab.extend(searchTweetsUserAct(user))     
        
    return results_tab


### PRINT IN A DATAFRAME TO EXPORT 2ND DATASET

In [ ]:
resultat2ndDt = search_usersTweets_info()

SecData = pd.DataFrame(resultat2ndDt)
SecData.to_csv('Second_Dataset.csv',encoding='utf-8-sig')
display (SecData)

## 3RD DATASET : SEARCH USERS INFORMATION FROM USER DICTIONNARY

In [5]:
def searchActor(name):
    
    apiAuth = authenticate()

    results=[]
    my_followers = []
    my_followers_id = []
    my_following = []
    

    for friend in tweepy.Cursor(apiAuth.get_friends, screen_name=name).items():
        my_following.append(friend.screen_name)
        
    for follower in tweepy.Cursor(apiAuth.get_followers, screen_name=name).items():
        my_followers.append(follower.screen_name)
      
    for followerid in tweepy.Cursor(apiAuth.get_follower_ids, screen_name=name).items():
        my_followers_id.append(followerid)


    user = apiAuth.get_user(screen_name = name)

    if not user is None :
        userId = user.id
        userName = user.screen_name
        userFollower = user.followers_count
        userFollow = user.friends_count
        location = user.location
        userVerified = user.verified
        userDescription = user.description
        userRt_count = user.status.retweet_count
        userFav_count = user.status.favorite_count

        line = {'ID': userId, 'NAME': userName, 'NB_FOLLOWER': userFollower, 'NB_FOLLOW': userFollow, 'LOCATION': location,
        'VERIFIED': userVerified, 'DESCRIPTION': userDescription, 'COUNT_RT': userRt_count, 'COUNT_FAV': userFav_count, 
        'NAME_FOLLOWERS': my_followers, 'NAME_FOLLOWING': my_following, 'ID_FOLLOWERS': my_followers_id}
        results.append(line)

    return results
   


def search_users_info():
    listOfActeurs = ["EmmanuelMacron"] # "EmmanuelMacron" OR "bcvorwerck"
    resultss = []
    
    for userName in listOfActeurs:
        resultss.extend(searchActor(userName))     
        
    return resultss


### PRINT IN A DATAFRAME TO EXPORT 3RD DATASET

In [ ]:
resultat3d = search_users_info()

ThrData = pd.DataFrame(resultat3d)
ThrData.to_csv('tabUserv.csv',encoding='utf-8-sig')
ThrData

Rate limit reached. Sleeping for: 772


In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

In [ ]:
#Fonction qui recherche tous les tweets des acteurs liés à l'écologie

resultat_string = []
def searchTweetsperActor():    
    analyze = []       
    
    for user in actor_dictionnary:
        analyze.extend(searchTweetsUserAct(user))
        
    return analyze

def arrangement_tweet():
    
    tweet_text_tab = []
    corpus = []
    ps = PorterStemmer()
    stop_words = stopwords.words('french')
    
    all_tweet_of_ecoactors = searchTweetsperActor()
    
    ecoactor_tweet_frame = pd.DataFrame(all_tweet_of_ecoactors)
    pd.options.display.max_colwidth = 10000000
    pd.options.display.max_rows = 1000000000


    tweet_text_tab.append(ecoactor_tweet_frame.TEXT)
    #print(tweet_text_tab)
    
#(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|[0-9]
    
    for i in range (len(tweet_text_tab)):        
        arrangedTweet = re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|[0-9]", ' ', str(tweet_text_tab[i]))       
        arrangedTweet = arrangedTweet.lower()        
        arrangedTweet = tokenize.sent_tokenize(str(tweet_text_tab[i]))        
        #arrangedTweet = arrangedTweet.split()
    
        #arrangedTweet = [x for x in arrangedTweet if not x in stop_words]
        #arrangedTweet = [ps.stem(x) for x in arrangedTweet]
        arrangedTweet = " ".join(arrangedTweet)
        
        
        corpus.append(arrangedTweet)
        
    sacdemots = []
       
    for i in range(len(corpus)):
        
        words = corpus[i].split()
        sacdemots.append(words)

        all_words = list(itertools.chain(*sacdemots))
        resultat_string.append(all_words)
        words_freq = collections.Counter(all_words)
        createtest = words_freq.most_common(300)
    
       
    return createtest

word_list_tweet = arrangement_tweet()


frame_list_word_count = pd.DataFrame(word_list_tweet, columns = ['Mots', 'Occurrences'])
#frame_list_word_count.to_csv("BIGFATAFRAMAA.csv", encoding="utf-8")
#display(frame_list_word_count)
#datacss.TEXT = datacss.TEXT.apply(lambda x : x.replace('\n', '\\n'))

#datacss.TEXT.to_csv("NEWGREATTEST.csv", encoding="utf-8",  index=False)

mask = frame_list_word_count['Mots'].isin(word_dictionnary_wordcloud)

#print(mask)



#display(frame_list_word_count)
#print(str(resultat_string))

final_tab = []

for each in word_dictionnary:
    print("DANS LE DICTIONNAIRE : " + each)
    
for each in resultat_string:
    for eachi in each:
        if eachi in word_dictionnary_wordcloud:
            print("DANS RESULTAT :", eachi)
            final_tab.append(eachi)
        
text_word = ' '.join(final_tab)    
print("SARAAAAAAAAAAH", text_word)
   #datacsv = pd.DataFrame(endo)
#datacsv.TEXT = datacsv.TEXT.apply(lambda x : x.replace('\n', '\\n'))

#testdesarahaenvoyer.to_csv('testaenvoyer2.csv', index=False, encoding='utf-8-sig', escapechar="\n")

#49
#96

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(15,10))
frame_list_word_count[mask].sort_values(by = 'Occurrences').plot.barh(x= 'Mots', y='Occurrences', ax=ax, color ="green")
plt.show()

In [ ]:
pip install wordcloud-1.8.1-cp39-cp39-win_amd64.whl

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

In [ ]:
def searchTweetsperActor():    
    analyze = []       
    
    for user in actor_dictionnary:
        analyze.extend(searchTweetsUserAct(user))
        
    return analyze
tweet_text_tab = []

    
all_tweet_of_ecoactors = searchTweetsperActor()
    
ecoactor_tweet_frame = pd.DataFrame(all_tweet_of_ecoactors)
tweet_frame = ecoactor_tweet_frame.TEXT.astype(str)
pd.options.display.max_colwidth = 10000000
pd.options.display.max_rows = 1000000000

tweet_text_tab.append(tweet_frame)
    
print(tweet_text_tab)

In [ ]:

wordcloud = WordCloud(background_color = 'white', max_words = 300, collocations=False, width=6000, height=4000).generate(text_word)
plt.imshow(wordcloud)
plt.axis("off")
plt.show()